**Data preprocessing**




In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('bankmkt.csv')

In [ ]:
df.shape

In [ ]:
df.info

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
# drop the duplicates
dups = df.duplicated()
print('before are there any duplicates : ', dups.any())

#df.drop_duplicates(inplace=True)
# reset indices after dropping rows
df=df.reset_index(drop=True)
print('after are there any duplicates : ', df.duplicated().any())



**Make a class for easier to experiment**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

In [ ]:
class FeedForwardNetwork:
    def __init__(self, dataset_file):
        self.dataset_file = dataset_file
        self.data = None
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        self.model = None

    def read_dataset(self):
        self.data = pd.read_csv(self.dataset_file)

    def preprocess_dataset(self):
        selected_features = ['age', 'marital', 'education', 'default', 'balance', 'housing', 'loan',
                             'contact', 'campaign', 'pdays', 'previous', 'poutcome']

        self.data = self.data.drop(['duration'], axis=1)

        # drop the duplicates
        dups = self.data.duplicated()
        print('before are there any duplicates : ', dups.any())
        self.data.drop_duplicates(inplace=True)

        # reset indices after dropping rows
        self.data=self.data.reset_index(drop=True)
        print('after are there any duplicates : ', self.data.duplicated().any())


        self.data['housing'] = self.data['housing'].map({'yes': 1, 'no': 0})
        self.data['loan'] = self.data['loan'].map({'yes': 1, 'no': 0})
        self.data['y'] = self.data['y'].map({'yes': 1, 'no': 0})



        target_variable = 'y'
        self.X = self.data[selected_features]
        self.y = self.data[target_variable]

        categorical_features = ['marital', 'education', 'default', 'housing', 'loan', 'contact', 'poutcome']
        le = LabelEncoder()
        for feature in categorical_features:
            self.X[feature] = le.fit_transform(self.X[feature])

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=42)

    def build_model(self, hidden_layers, hidden_nodes):
        self.model = Sequential()
        self.model.add(Dense(hidden_nodes, input_dim=self.X_train.shape[1], activation='relu'))

        for _ in range(hidden_layers):
            self.model.add(Dense(hidden_nodes, activation='relu'))

        self.model.add(Dense(1, activation='sigmoid'))

    def train_model(self, epochs, learning_rate):
        optimizer = Adam(learning_rate=learning_rate)
        self.model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        early_stopping = EarlyStopping(patience=10, restore_best_weights=True)  # Early stopping to prevent overfitting

        self.model.fit(self.X_train, self.y_train, epochs=epochs, batch_size=32, verbose=1,
                       validation_data=(self.X_test, self.y_test), callbacks=[early_stopping])

    def evaluate_model(self):
        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        print(f'Test Loss: {loss:.4f}')
        print(f'Test Accuracy: {accuracy:.4f}')
        return loss, accuracy

In [ ]:
# Initialize the feed-forward network
network = FeedForwardNetwork('bankmkt.csv')

In [ ]:
# Read and preprocess the dataset
network.read_dataset()
network.preprocess_dataset()

In [ ]:
experimental_df=pd.read_csv('experimental.csv')

In [ ]:
experimental_df.head(100)

In [ ]:
for i, j in experimental_df.iterrows():
    print(j['Architecture'])
    # Set the network parameters:
    hidden_layers = j['HiddenLayers'].astype(int)
    hidden_nodes = j['NodesperLayer'].astype(int)
    epochs = j['Epochs'].astype(int)
    learning_rate = j['LearningRate']

    # Build and train the model
    network.build_model(hidden_layers, hidden_nodes)
    network.train_model(epochs, learning_rate)

    # Evaluate the model

    experimental_df.at[i,'Loss'], experimental_df.at[i,'Accuracy'] =network.evaluate_model()




In [ ]:
experimental_df.head(100)